# Guide for downloading images  

## Token information  

Here, we used Autorized token given by https://developers.arcgis.com/dashboard  
Please make ArcGIS developers account. It's free!

## Daytime imagery  

There are three codes to run for downloading images : ```metadata_collection.py```, ```image_downloader_shuf.py```, ```image_downloader_from_csv.py```


### Metadata collection  

```metadata_collection.py``` 
+ input
    * necessary 
        - zoom level (--zl)
        - nation (--nation)
        - minor (--minor)
        - data set (--dset)
        - data collection name (--dcoll)
        - Autorized token (--token)
    * necessary, naming variables
        - name of output \[z,y,x\] for each district, saved as .json (--ozyx)
        - name of output demographic data referring data set and data collection, saved as .csv (--dset)
    * optional
        - start, end points when you want to split the job (--start, --end respectively)  
        
+ output
    * \[z,y,x\] for each district, saved as .json
    * Demographic data referring data set and data collection, saved as .csv
    


#### Notes  

For *nation*, you can check the nation code of each country at  
https://developers.arcgis.com/rest/geoenrichment/api-reference/geoenrichment-coverage.htm  

For *minor*, *dset*, *dcoll*, you can check the inputs from  
https://geoenrich.arcgis.com/arcgis/rest/services/World/geoenrichmentserver/Geoenrichment/Countries

You can even observe specific country that you selected. In case of South Korea, nation code is __KOR__. The URL  
https://geoenrich.arcgis.com/arcgis/rest/services/World/geoenrichmentserver/Geoenrichment/Countries/KOR?f=pjson  
returns information at South Korea

#### code example  
On Uganda(UGA) with zoom level 15:  
```python3 metadata_collection.py --zl 15 --ozyx ozyx_ug_district_z15 --odat odat_ug_district_z15 --nation UGA --minor UG.Districts --dset UGA_MBR_
2019 --dcoll Gender --ttok {token}
```
        

### Daytime image download  

```image_downloader_shuf.py```, ```image_downloader_from_csv.py```

Actually, both codes are doing same thing : they call the images to download from .json file which is generated by ```metadata_collection.py```.  
```image_downloader_shuf.py``` first performs random shuffle for the districts that need to be download soon. And it saves this shuffled areas are memorized at *shuffled.csv*  
If you run this, it sometimes report error message due to the connection problem.
Then, ```image_downloader_from_csv.py``` makes us to continue the terminated process by referring *shuffled.csv*. 


```image_downloader_shuf.py```  
+ input
    * necessary
        - ozyx .json file path from metadata_collection.py (--zyx)
        - odat .csv file path from metadata_collection.py (--dat)
        - Autorized token (--token)
    * necessary, naming variables
        - directory name to save whole images (--odir)
        
+ output
    * Images saved as ... odir/{district_num}/{y_x.png}

```image_downloader_from_csv.py```  
+ input
    * necessary
        - ozyx .json file path from metadata_collection.py (--zyx)
        - shuffled.csv file path from image_downloader_shuf.py (--shufdat)
        - the number of districts to skip (--skip)
        - Autorized token (--token)
    * necessary, naming variables
        - directory name to save whole images. This would be same as odir value of image_downloader_shuf.py (--odir)
        
+ output
    * Images saved as ... odir/{district_num}/{y_x.png}

### Daytime Ground-Truth statistics

The code ```popden_fb.py``` reads the population density file from  https://data.humdata.org/organization/facebook  
You may download .csv file from above URL by searching target country.
It contains popuation density for every second X second (~30m x 30m), which is even finer than zoom level 20. This code aggregates the population density observed within our target zoom level for leveraging ground truth for each grid.  

```popden_fb.py```

+ input
    * necessary
        - zoom level (--zl)
        - facebook population density file path (--fbcsv)
    * necessary, naming variables
        - output .csv file name (--ocsv)
        
+ output
    * population density per each y_x.png images, as {ocsv}.csv file
    


### Nighttime image download  



The code ```earth_at_night.py``` downloads the nighttime imagery with maximum zoom level 9, and crop it with the size of daytime imagery. So it requires daytime imagery file path as one of the inputs. For an intermediate step, this also returns the original nightlight images that are not cropped.

```earth_at_night.py```   
+ input
    * necessary
        - daytime satellite imagery directory path (--ddir)
        - zoom level (usually 9)(--zl)
        - zoom level difference between nightime and daytime. Ex) 15-9 = 6(--zdiff)
        - nation code (--nation)
    * necessary, naming variables
        - directory name for original nightlight images (--ondir)
        - directory name for cropped nightlight images (--cndir)
        
+ output
    * Images saved as ondir/{y_x.png} (Nighttime sat images)
    * images saved as cndir/{district num}/{y_x.png} (Cropped nighttime sat images)
    


### Nightlight image data evaluation

The code ```night_eval.py``` get input of nightlight imageries with directory path,  
where the imageries are saved as {nightlight_dir}/{district}/{y_x.png}  
The code evaluate each nightlight image by taking pixel value as grayscale, and aggregate them as one value per each y_x.png image


```night_eval.py ```
+ input
    * necessary
        - target nightlight imagery directory(--tdir)
        - Facebook population density csv file path (--odir)
        - ouput csv file name (--ocsv)
        
+ output
    * aggregated nightlight density per each y_x.png images, as {ocsv}.csv file
